## A Tutorial based from the paper : "An Evaluation of Naive Bayesian Anti-Spam Filtering

This is a tutorial to replicate the process of the paper entitled: "An Evaluation of Naive Bayesian Anti-Spam Filtering. The objective of the paper is to filter spam emails using a Naive Bayes Techinique

## Preparing our Data

The dataset provided to us has 4 directories [bare, lemm, lemm_stop, stop] with subdirectories of 10 parts. Where 9 were used as training set and 1 reserved for testing for every repetition. To reduce random variation, a ten cross-validation was done yielding the ten subdirectories.


To form a list of features to finally use in predicting classification using the Naive Bayes theorem, a common feature selection method is done by computing the Mutual Information (MI) of term t and class c 

where, 
- <i>t</i> is defined to be a word attribute and;
- classified either as <i>c</i> = spam or not spam. 

We are given a description <i>d ∈ X</i> of a document, where <i>X</i> is the document space; and a fixed set of classes <i>C = {spam, legitimate}</i>

Before training our data to filter out spam, we need to find the features that will be used, the paper used the words found in the corpus as the features of the classifier. We created a whole tutorial on feature extraction alone, refer to another notebook named: <b><i>Jupyter Feature Extraction : Feature Extraction.ipynb</i></b>.

It will generate a csv file that has:
1. Term
2. Number of legitimate emails that word occurred in
3. Number of spam emails that word occurred in
4. Mutual Information Count

After reading the feature extraction tutorial, you already have the list of all the terms in each corpus with their corresponding Mutual Information score, the following function will extract n-terms with the highest <i>MI</i>. The you/system decide how many features will be used. 

The paper used 50 - 700 features, stepping by 50. 

In [1]:
import pandas as pd
numMI = [50,100,150,200,250,300,350,400,450,500,550,600,650,700]
corpus = ['bare','lemm', 'lemm_stop', 'stop']

for corp in corpus:
    for num in numMI:
        termMIList = pd.read_csv("Features/"+corp+"/"+corp+"termMI.csv", index_col = 0)
        terms = pd.DataFrame(termMIList['Term'].head(n=num).tolist()).to_csv("MI/"+corp+"/"+str(num)+"terms.txt", header = None, index = None)
print ("Done, check MI folder")

Done, check MI folder


We then create the train and test set split by passing the extracted <i>.txt</i> file to the function below that automatically walks through the directory given to it and transfers the contents of the text file into a list called <i>dir_dataset</i>, where the function <i>parse_subdirectories</i> returns it.

In [12]:
import os

# eg. passed directory = 'Emails/bare'
# reads its subdirectories and files into a list of lists
def parse_subdirectories(directory):
    for path, subdirs, files in os.walk(directory):
        dir_dataset = []
        for filename in files:
            f = os.path.join(path,filename)
            subdir_content = []
            with open(f,'r') as file_content:
                content = file_content.read()
                subdir_content.append(content)
        dir_dataset.append(subdir_content)
    return dir_dataset

print ("done")

done


We then split this list data into 90% training set and 10% data for accuracy testing of Naive Bayes prediction. Let's call these list variables: <i>train_set</i> and <i>test_set</i>.

In [11]:
import random

def split_dataset(dataset, n_split):
    train_set = []
    data_copy = list(dataset)
    while len(train_set) < int(len(dataset)*n_split):
        pointer = random.randrange(len(data_copy))
        train_set.append(data_copy.pop(pointer))
    return [train_set,data_copy]

print ("done")

done


## Classification of Email/Documents 

In order to classify the documents whether it is of category spam or legit. To do this, we ought to apply the formula of Bayes which needs the probability of the extracted terms/features.

The classifier we will be building will use "supervised learning". Now that we have the a list of features, its association with all messages using the term matrix, this will form the basis of the classifier.

Once we have associated the various words with our two classifications (spam and legit), we can calculate the probability that a given word belongs to either spam or legit category. For instance, the probability that the word "vintage" appears in a spam message is much higher than the probability it appears in a legitimate email.

For example, once we have trained our classifier using 200 documents, 100 are spam and 100 are legit. If word "vintage" appears in 25 spam documents, but only 5 legit documents. The probability, then, that the word "vintage" classifies as a spam document is calculated:

$$P("vintage" | spam) = (.25 * .5) / ((.25 * .5) + (.05 * .5)) = .83, or 83%.$$

The ".25" and ".05" are the percentage of documents containing the word money that are spam and ham respectively. The ".5" is the interesting number and is the percentage of documents that are spam or legit. Since we have classified 100 of each, the total number of documents is 200, and it is overall 50% likely that a document is spam.

By combining the probabilities for all the words in a document, it is possible to get an overall view of the likelihood a document is either spam or legit.

We can now proceed in computing for the probability of classifiying it as a spam or not with the equation:

$$\frac{P(C=spam|\vec{X} = \vec{x})}{P(C=legitimate | \vec{X} = \vec{x})} > \lambda$$

The equation tells us that we need the vectors from the term matrix (counts) which indicates association between the features and documents in order to assess if it is a spam or not.

Let's create function/s for these.

In [ ]:
import math
from decimal import *

getcontext().prec = 256
getcontext().rounding = ROUND_UP

# A = P(X=x|C=c)
# P(A|B), is equal to P(AB)/P(B).
# P(A) = (Total number of times x occurred/total number of term occurrence in the corpus)
# P(B) = (Total number of email c in the corpus /total number of documents in the corpus)
#termProb = total number of times x occurred  in document


def computeTermGivenClass(probClass, terms, totalTerms):
    prob = Decimal(1.0)
    for x in terms:
        a = (Decimal(x)/Decimal(totalTerms))
        prob = (Decimal(prob) * (Decimal(a)/Decimal(probClass)))
    
    return prob

#B = P(X=x) = (Total number of times x occurred in the corpus)/(Total number of word occurrence in the corpus)
#C = P(C=c) = (Total number of documents that are c in the corpus)/(Total number of documents in the corpus)
          
def computeProbability(isComputingSpam, totalDoc, spamTerms, legitTerms, totalTerms, totalLegitCount, totalSpamCount):
      
    probSpam = (totalSpamCount/totalDoc)
    probLegit = (totalLegitCount/totalDoc)
    
    givenSpam =  (Decimal(probSpam) * computeTermGivenClass(probSpam, spamTerms, totalTerms))
    givenLegit = (Decimal(probLegit) * computeTermGivenClass(probLegit, legitTerms, totalTerms))
    
    if isComputingSpam == True:
        numerator =  givenSpam
    else:
        numerator =  givenLegit
            
    denominator = Decimal(givenSpam) + Decimal(givenLegit)
    try:
        probClass = Decimal(numerator/denominator)
    except:
        probClass = 0
    return probClass

In [ ]:
import re
def extractWords(filepath):
    file = open(filepath, 'r')
    # .lower() returns a version with all upper case characters replaced with lower case characters.
    text = file.read().lower()
    file.close()
    # replaces anything that is not a lowercase letter, a space, or an apostrophe with a space:
    text = re.sub('[^a-z]+', " ", text)
    words = list(text.split())
    
     # remove duplicate words in the list
    words = list(set(words))
    # removes words that are less than 4 letters/characters
    words =  [i for i in words if len(i) >= 4] 
    return words;

def findTermCounts(terms, docTerms):
    return terms.loc[terms['Term'].isin(docTerms)]

In [ ]:
def classifyDocs(corp, terms, spamCount, legitCount, totalTerms,
             fileList, threshold, numFeatures):
    
    totalDoc = corp.totalEmailCtr
    totalLegitCount = corp.legitEmailCtr
    totalSpamCount = corp.spamEmailCtr

    predicted = []
    for filepath in fileList:   

        docTerms = extractWords(filepath)
        rowTerms = findTermCounts(terms, docTerms)
        PCSpam = Decimal(computeProbability(True, totalDoc, spamCount, legitCount, 
                                            totalTerms, totalLegitCount, totalSpamCount))
        PCLegit = Decimal(computeProbability(False, totalDoc, spamCount, legitCount, 
                                            totalTerms, totalLegitCount, totalSpamCount))
        
        try:
            ifSpam = PCSpam/PCLegit
        except:
            ifSpam = 0
            
        if  ifSpam > threshold:
            predicted.append(0)
        else:
            predicted.append(1)
    
    return predicted

In [ ]:
# create a class for the the Corpus data, it will store the total number of emails in the corpus, 
# along with the total number of spam and legit emails
class CorpusData: 
    corpusName = ""
    totalEmailCtr = 0
    spamEmailCtr = 0
    legitEmailCtr = 0

    def __init__(self, corpusName, totalEmailCtr, spamEmailCtr, legitEmailCtr):
        self.corpusName = corpusName
        self.totalEmailCtr = totalEmailCtr
        self.spamEmailCtr = spamEmailCtr
        self.legitEmailCtr = legitEmailCtr   

In [ ]:
def combineFiles(corp):
    #for each subdirectory in a corpus (folders - part 1 - 10)
    fileList = []
    rootdir = "Emails/"+corp
    actualClass = []
    for subdir, dirs, files in os.walk(rootdir):
    #for each file in a folder
        for file in files:
            filepath = subdir + "/" + file
            fileList.append(filepath)
            
            if pattern.match(file): 
                    actualClass.append(1)
            else:
                    actualClass.append(0)
    
    return fileList, actualClass

In [ ]:
def writeResults(actual, predicted, corp, featureNum, thresh):
    results = pd.DataFrame(
        {'Actual': actual,
         'Predicted' : predicted,
        })

    #save the Term MI to CSV (so we can access it later)
    results.to_csv("Classified/"+corp.corpusName +"/"+str(featureNum)+"/"+str(thresh)+"_results.csv")
    print("File Saved: ", corp.corpusName , featureNum, thresh)

In [ ]:
# total emails, total spam emails, total legit emails
bare = CorpusData("bare", 2515, 304, 2211)
lemm = CorpusData("lemm", 2776, 452, 2324)
lemm_stop = CorpusData("lemm_stop", 2609, 281, 2409)
stop = CorpusData("stop", 2341, 481, 1860)


numFeatures = [50,100,150,200,250,300,350,400,450,500,550,600,650,700]
# 0.5 - 1, 0.9 - 9, 0.999 - 999
threshold = [0.5, 0.9, 0.999]

In [ ]:
def classify(corp):
    print ("Classifying", corp.corpusName)
    #load the vocabulary/word/term list for the entire corpus from file
    corpusTerms = pd.read_csv("Features/"+corp.corpusName+"/"+corp.corpusName+"termMI.csv", index_col = 0)
    totalLegitTerms = corpusTerms['LegitCount'].sum(axis=0)
    totalSpamTerms = corpusTerms['SpamCount'].sum(axis=0)
    totalTerms = totalLegitTerms + totalSpamTerms
    
    filepathList, actualClass = combineFiles("bare")
    for num in numFeatures:  
        terms = pd.read_csv("MI/"+corp.corpusName+"/"+str(num)+"terms.csv", index_col = 0)
        print ("Features: ", num) 
        
        spamCount = terms['SpamCount'].tolist()
        legitCount = terms['LegitCount'].tolist()
        
        for t in threshold:
            print ("Threshold: ", t)
            predClass = classifyDocs(corp, terms, spamCount, legitCount, totalTerms,
                                 filepathList, t, num)
            writeResults(actualClass,predClass, corp, num, t)

We can now classify a corpus like so:

In [ ]:
classify(bare)

In [ ]:
print ("Classifying", bare.corpusName)
#load the vocabulary/word/term list for the entire corpus from file
corpusTerms = pd.read_csv("Features/"+bare.corpusName+"/"+bare.corpusName+"termMI.csv", index_col = 0)
totalLegitTerms = corpusTerms['LegitCount'].sum(axis=0)
totalSpamTerms = corpusTerms['SpamCount'].sum(axis=0)
totalTerms = totalLegitTerms + totalSpamTerms

filepathList, actualClass = combineFiles("bare")
num = 400 
terms = pd.read_csv("MI/"+bare.corpusName+"/"+str(num)+"terms.csv", index_col = 0)
print ("Features: ", num) 

spamCount = terms['SpamCount'].tolist()
legitCount = terms['LegitCount'].tolist()

t = 0.5
print ("Threshold: ", t)
predClass = classifyDocs(bare, terms, spamCount, legitCount, totalTerms,
                     filepathList, t, num)
writeResults(actualClass,predClass, bare, num, t)

In [13]:
import os

#subdirectory : 'Emails/bare
def countDocuments(directory):
    for path, subdirs, files in os.walk(directory):
        for subdir in subdirs:
            dir_path = os.path.join(path,subdir)
            docCount = len([name for name in os.listdir(dir_path)])
            spamCount = 0
            legitCount = 0
            for name in os.listdir(dir_path):
                if name.startswith("spm"):
                    spamCount += 1
                else:
                    legitCount += 1
    return docCount,spamCount,legitCount

print(countDocuments('Emails/bare'))
print(countDocuments('Emails/lemm'))
print(countDocuments('Emails/lemm_stop'))
print(countDocuments('Emails/stop'))

(289, 48, 241)
(279, 48, 231)
(289, 48, 241)
(289, 48, 241)


## Evaluation

After classifying the documents whether it is spam or legitimate, they have been done only for testing purposes, up until now. The last part of this tutorial covers about the Performance Evaluation. Check out and open <b><i>Performance Evaluation.ipynb</i></b> for the tutorial of this part!

# And then, We're Done! :)